In [1]:
%pylab inline
import pandas, json
reviews = pandas.DataFrame(json.loads(l) for l in open('yelp/review.json'))

Populating the interactive namespace from numpy and matplotlib


In [18]:
reviews['useful'] = reviews['votes'].apply(lambda v: v['useful'])

In [3]:
ytract = pandas.DataFrame(json.loads(l) for l in open('business_track.json'))
business = pandas.DataFrame(json.loads(l) for l in open('yelp/business.json'))

In [6]:
cats = []
for d in (json.loads(l) for l in open('yelp/business.json')):
    for c in d['categories']:
        cats.append(dict(business_id = d['business_id'], category=c))
cats = pandas.DataFrame(cats)

In [14]:
cats = cats.merge(ytract)

counts_by_geocat.to_csv('counts_by_geocat.csv')

In [23]:
review_cats = reviews[['business_id', 'stars', 'useful']].merge(cats)

In [28]:
review_cats['wstars'] = review_cats.stars * (review_cats.useful + 1)



In [103]:
stars_by_geocat = review_cats[['stars']].groupby([review_cats.category, review_cats.GISJOIN]).mean()
stars_by_geocat = stars_by_geocat.unstack(level=0).dropna(how='all').dropna(axis=1, thresh=99)
stars_by_geocat.columns = stars_by_geocat.columns.get_level_values(1)
stars_by_geocat.reset_index(inplace=True)
stars_by_geocat.to_csv('stars_by_geocat.csv')

wstars_by_geocat = review_cats[['wstars']].groupby([review_cats.category, review_cats.GISJOIN]).mean()
wstars_by_geocat = wstars_by_geocat.unstack(level=0).dropna(how='all').dropna(axis=1, thresh=99)
wstars_by_geocat.columns = wstars_by_geocat.columns.get_level_values(1)
wstars_by_geocat.reset_index(inplace=True)
wstars_by_geocat.to_csv('wstars_by_geocat.csv')

counts_by_geocat = cats[['business_id']].groupby([cats.category, cats.GISJOIN]).count()
counts_by_geocat.columns = ['business_count']
counts_by_geocat = counts_by_geocat.unstack(level=0).dropna(how='all').dropna(axis=1, thresh=99)
counts_by_geocat.columns = counts_by_geocat.columns.get_level_values(1)
counts_by_geocat.reset_index(inplace=True)
counts_by_geocat.to_csv('counts_by_geocat.csv')

In [68]:
data = pandas.read_csv('zip/rus2/nhgis0002_ds201_20135_2013_tract.csv', skiprows=[1])

In [75]:
data['pc_income'] = data['UJAE001']
data['employed_percent'] = data['UJ8E002'] / data['UJ8E001']
data['median_age'] = data['UEFE001']
data['median_gross_rent'] = data['UL9E001']
data['median_family_income'] = data['UINE001']
econ = data[['GISJOIN', 'pc_income', 'employed_percent', 'median_age', 'median_gross_rent', 'median_family_income']]

In [125]:
wcorrs = econ.merge(wstars_by_geocat).corr().unstack().reset_index()
wcorrs.columns = ['econ', 'cat', 'r']
wcorrs = wcorrs[wcorrs.econ.apply(lambda e: e.islower())]
wcorrs = wcorrs[wcorrs.cat.apply(lambda e: not e.islower())]
wcorrs[wcorrs.r.apply(lambda v: abs(v) > .3)]

,econ,cat,r
98,pc_income,Hotels,0.303225
223,employed_percent,Car Rental,-0.308624


In [126]:
scorrs = econ.merge(stars_by_geocat).corr().unstack().reset_index()
scorrs.columns = ['econ', 'cat', 'r']
scorrs = scorrs[scorrs.econ.apply(lambda e: e.islower())]
scorrs = scorrs[scorrs.cat.apply(lambda e: not e.islower())]
scorrs[scorrs.r.apply(lambda v: abs(v) > .3)]

,econ,cat,r
842,median_family_income,Hotels,0.329872


In [127]:
ccorrs = econ.merge(counts_by_geocat).corr().unstack().reset_index()
ccorrs.columns = ['econ', 'cat', 'r']
ccorrs = ccorrs[ccorrs.econ.apply(lambda e: e.islower())]
ccorrs = ccorrs[ccorrs.cat.apply(lambda e: not e.islower())]
ccorrs[ccorrs.r.apply(lambda v: abs(v) > .3)]

,econ,cat,r
5,pc_income,Accessories,0.314696
14,pc_income,Asian Fusion,0.318272
21,pc_income,Bakeries,0.333604
27,pc_income,"Beer, Wine & Spirits",0.373052
33,pc_income,Buffets,0.357909
55,pc_income,Desserts,0.384839
56,pc_income,Diners,0.311126
92,pc_income,Hobby Shops,0.334230
97,pc_income,Hot Dogs,0.404527
106,pc_income,Jewelry,0.407347
